<a href="https://colab.research.google.com/github/FarhanFansuri/Anjing-vs-Kucing-Image-Augmentation/blob/main/cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Library

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset, info = tfds.load('cats_vs_dogs', with_info = True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteGFMS7L/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [4]:
label_names = info.features['label'].names

['cat', 'dog']

In [6]:
for i, data in enumerate(dataset['train']):
  image, label = data
  foldername = './cats_vs_dogs/train/{}'.format(label_names[label])
  os.makedirs(foldername,exist_ok = True)
  filename = foldername + '/{}_{}.jpg'.format(label_names[label],i)
  tf.keras.preprocessing.image.save_img(filename, image.numpy())
